In [1]:
#Importing important libraries.
#Import sys and scipy.io for Linux.
#import sys
#from scipy.io import arff
#import arff for Windows.
import arff
import math
import pandas as pd
import numpy as np
from numpy import random
from numpy import linalg as LA

In [119]:
########Lodaing of data in Windows.#####################
########Training data set (but to delete).############################
#raw_data = arff.load(open('lymph_train.arff')) # Loading the training datasets
raw_data = arff.load(open('vote_train.arff')) # Loading the training datasets
# Creating a list of the features
feature_list = [] 
for x in raw_data['attributes']:
    feature_list.append(x[0])
training_data = pd.DataFrame(np.array(raw_data['data']), columns = feature_list) # Saving as a Pandas Dataframe
training_data = training_data.apply(pd.to_numeric, errors='ignore') # Converting the releavent columns to numeric
###################Testing data set (but to delete).###################################
#raw_data = arff.load(open('lymph_test.arff')) # Loading the training datasets
raw_data = arff.load(open('vote_test.arff')) # Loading the training datasets
# Creating a list of the features
feature_list = [] 
for x in raw_data['attributes']:
    feature_list.append(x[0])
testing_data = pd.DataFrame(np.array(raw_data['data']), columns = feature_list) # Saving as a Pandas Dataframe
testing_data = testing_data.apply(pd.to_numeric, errors='ignore') # Converting the releavent columns to numeric
#training_data

In [3]:
#Initializing key variables and dictionaries for the code.
P_Y = {}
P_X_given_Y_positive = {}
P_X_given_Y_negative = {}

In [4]:
#Determining the probability of class P(Y).
def Prob_Y(data, feature):
    classification_frequency = {}
    Probability = {}
    classification_frequency = data[feature].value_counts().to_dict()
    for i in classification_frequency:
        Probability[i] = classification_frequency[i]/len(data)
    return Probability

In [11]:
#Determining the conditional probability P(X|Y).
def Prob_X_condition_Y(data, data_split, feature):
    classification_frequency = {}
    Probability = {}
    classification_frequency = data_split[feature].value_counts().to_dict() #Finding the frequency of sub-features in each column.
    feature_data = data[feature].unique() #Finding the unique elements (sub - features) in a column.
    for i in feature_data: #Determining the probability for each of the sub - feature in the column given Y (split data).
        check = 0
        for j in classification_frequency:
            if (i == j):
                Probability[i] = classification_frequency[j]/len(data_split)
                check+=1                
        if (check == 0): #If there're no elements corresponding to a sub - feature, then the probability is zero. 
            Probability[i] = 0.0           
    return Probability

In [13]:
#Determining the conditional probability P(X1, X2|Y).
def Prob_X1_X2_condition_Y(data_split, feature1, feature2, sub_feature1, sub_feature2):
    filtered_data = data_split[(data_split[feature1] == sub_feature1) & (data_split[feature2] == sub_feature2)] #Applying the filter and reducing the data set.
    Count_of_filtered_data = len(filtered_data) #Determining the number of data points belonging to (x1, x2).
    if (Count_of_filtered_data != 0):
        Probability = Count_of_filtered_data/len(data_split) #Determining P(X1, X2|Y) for non-zero data points.             
    else:           
        Probability = 0.0  #If no data points belonging to (x1, x2), the assigned probability is 0.         
    return Probability

In [14]:
#Training of the dataset for naive Bayes algorithm.
#Determining the conditional probability for the entire dataset.
def Training_naive(data):
    prediction_data = data.copy()
    P_Y = Prob_Y(data, 'class')
    class_labels = data['class'].unique()
    data_positive = data[(data['class'] == class_labels[0])].reset_index(drop = True) #Getting the data corresponding to positive label.
    data_negative = data[(data['class'] == class_labels[1])].reset_index(drop = True) #Getting the data corresponding to negative label.
    #Training the positive values in dataset.
    for i in feature_list:
        if i == 'class':
            break
        P_X_given_Y_positive[i] = Prob_X_condition_Y(data, data_positive, i)
    #Training the negative values in the dataset.   
    for j in feature_list:
        if j == 'class':
            break
        P_X_given_Y_negative[j] = Prob_X_condition_Y(data, data_negative, j)
    return P_X_given_Y_positive, P_X_given_Y_negative

In [15]:
#Prediction for naive Bayes algorithm.
def Prediction_naive(data):
    Prediction_comparison  = {}
    class_labels = data['class'].unique()
    sum = 0.0 #Counter variable for number of current predictions.
    print('*****************************************************************')
    print('***Comparison of actual versus predicted value for Naive Bayes***')
    print('*****************************************************************')
    for i in range(0, len(data)): #Predicting each row in the testing data set.
        accumulated_log_of_positive_probabilities, accumulated_log_of_negative_probabilities = 0, 0 #Initializing the sum of accumulated probability with 0.
        row_data = data.iloc[i, :].to_dict() #Reading each row of the data set.
        for i in row_data:
            for j in P_X_given_Y_positive:        
                if (i == j): #Determining the accumulated probability in favor of positive label.
                    accumulated_log_of_positive_probabilities+= math.log2 (P_X_given_Y_positive[j][row_data[i]]+1)
            for k in P_X_given_Y_negative:
                if (i == k): #Determining the accumulated probability in favor of negative label.
                    accumulated_log_of_negative_probabilities+= math.log2 (P_X_given_Y_negative[k][row_data[i]]+1)    
        if (accumulated_log_of_positive_probabilities > accumulated_log_of_negative_probabilities): #Checking which accumulated probability is greater.
            prediction = class_labels[0] #Assigning the predicted label accordingly.
        else: 
            prediction = class_labels[1] #Assigning the predicted label accordingly.
        print('Prediced:', prediction, 'Actual: ', row_data['class'])
        if (row_data['class'] == prediction): #If prediction is right.
            sum+= 1 #Increase the sum by 1.
    accuracy = sum/len(data) #Determining the accuracy of the Naive Bayes algorithm.
    print('*****************************************************************')
    print('Number of data points:', len(data))
    print('Number of accurate results: ', sum)
    print('Accuracy is: ', accuracy)

In [71]:
#Determining weights for each edges using TAN algorithm.
def Weight (X_1, X_2, data):
    I_X1_X2 = 0
    for y in data['class'].unique(): #Loop over y belonging to labels.
        data_split = data[(data['class'] == y)].reset_index(drop = True)
        Prob_X1_condition_Y = Prob_X_condition_Y(data, data_split, X_1)
        Prob_X2_condition_Y = Prob_X_condition_Y(data, data_split, X_2)
        for i in data[X_1].unique(): #Loop over x1 belonging to X1 
            for j in data[X_2].unique(): #Loop over x2 belonging to X2 
                Z = Prob_X1_X2_condition_Y(data_split, X_1, X_2, i, j) #Determining I(X1, X2| Y) which is the weight for edges for MST. 
                I_X1_X2+= Prob_Y(data, 'class')[y]*Z*(math.log2((Z+0.0001)/ ((Prob_X1_condition_Y[i]*Prob_X2_condition_Y[j])+0.00001))) #Laplace estimates.
    return I_X1_X2 #Returning the weight.

In [72]:
#Determing the weight for the MST.
def Weight_collection(data):
    from collections import defaultdict #Importing library for 2D dicts.
    weight = defaultdict(dict) #Intializing weight.
    V = feature_list.copy() #Finding the vertices for the graph.
    del V[-1] #Dropping the last column of 'class' from the set of vertices.
    for i in V: #Iterating over the set of vertices.
        for j in V: #Iterating over the set of vertices.
            if (i!=j):
                weight[i][j] = Weight(i, j, data)
            else:          
                weight[i][j] = -1           
    return weight

In [73]:
#Minimum spanning tree (MST).
def MST(weight):
    V = feature_list.copy() #Initializing V.
    del V[-1] #Removing the last column 'class' from V.
    V_new = dict() #Initializing empty vertices directory.
    E_node = [] #Initializing empty edge list.
    E_new = {}  #Initializing empty edge directory.
    V_new[V[0]] = V[0] #Assigning the first feature in the dataset as the root of the graph.
    selected_origin = ""
    selected_destination = ""
    #Starting the maximum spanning tree.
    while (set(list(V_new.keys()))!=set(V)):
        maximum = 0
        update = 0
        for i in list(V_new): #Running the loop over all the nodes in V_new.
            for j in weight:   #Finding the selected node in weight matrix.
                if (i == j):
                    for k in weight[j]: #Determining the node to go from the selected node.
                        if (weight[j][k] > maximum): #Checking if the weight is maximum to visit that node.
                            if k in V_new: #Checking if the node has already been selected or not.
                                update+=1 #If the node has already been selected, we don't consider in the present loop.
                            if (update == 0): #If the node has not been selected, update the best-so-far node selection.
                                maximum = weight[j][k]
                                selected_origin = i
                                selected_destination = k
                        update = 0
        V_new[selected_destination] = selected_destination #Adding the node v to V_new.
        E_node = [selected_origin, selected_destination] #Adding the edges e to the set of edge E_new.
        if selected_origin in E_new.keys():  #Adding the edges in the for of list to create a tree.
            E_new[selected_origin].append(selected_destination)  
        else:  
            E_new[selected_origin] = [selected_destination] 
    return E_new

In [113]:
#Determining the conditional probability P(X|X1, Y).
def Prob_X_condition_X1_Y(data_split, featureX, featureX1, sub_featureX, sub_featureX1):
    data_split = data_split[(data_split[featureX1] == sub_featureX1)].reset_index(drop = True)
    filtered_data = data_split[(data_split[featureX] == sub_featureX)] #Applying the filter and reducing the data set.
    Count_of_filtered_data = len(filtered_data) #Determining the number of data points belonging to (x1, x2).
    if (Count_of_filtered_data != 0):
        Probability = Count_of_filtered_data/len(data_split) #Determining P(X1, X2|Y) for non-zero data points.             
    else:           
        Probability = 0.0  #If no data points belonging to (x1, x2), the assigned probability is 0.         
    return Probability

In [114]:
featureX = feature_list[1]
featureX1 = feature_list[12]
sub_featureX = 'y'
sub_featureX1 = 'y'
data = training_data
class_labels = data['class'].unique()
data_split = data[(data['class'] == class_labels[0])].reset_index(drop = True)
p = Prob_X_condition_X1_Y(data_split, featureX, featureX1, sub_featureX, sub_featureX1)
print (p)

0.5126050420168067


In [117]:
featureX1

'superfund-right-to-sue'

In [115]:
R = feature_list.copy()
R

['handicapped-infants',
 'water-project-cost-sharing',
 'adoption-of-the-budget-resolution',
 'physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups-in-schools',
 'anti-satellite-test-ban',
 'aid-to-nicaraguan-contras',
 'mx-missile',
 'immigration',
 'synfuels-corporation-cutback',
 'education-spending',
 'superfund-right-to-sue',
 'crime',
 'duty-free-exports',
 'export-administration-act-south-africa',
 'class']

In [ ]:
data = training_data
R = feature_list.copy()
from collections import defaultdict #Importing library for 2D dicts.
Probability_positive_label = defaultdict(dict) 
Probability_negative_label = defaultdict(dict)
y = 'republican'
for i in tree:
    data_split = data[(data['class'] == y)].reset_index(drop = True) 
    Probability_positive_label[tree[i]][i] = Prob_X_condition_X1_Y(data_split, tree[i], i, sub_featureX, sub_featureX1)



In [105]:
tree

{'aid-to-nicaraguan-contras': ['anti-satellite-test-ban',
  'adoption-of-the-budget-resolution'],
 'anti-satellite-test-ban': ['export-administration-act-south-africa'],
 'el-salvador-aid': ['aid-to-nicaraguan-contras',
  'mx-missile',
  'physician-fee-freeze'],
 'export-administration-act-south-africa': ['immigration'],
 'handicapped-infants': ['religious-groups-in-schools'],
 'religious-groups-in-schools': ['el-salvador-aid',
  'crime',
  'superfund-right-to-sue',
  'education-spending'],
 'superfund-right-to-sue': ['duty-free-exports', 'water-project-cost-sharing'],
 'water-project-cost-sharing': ['synfuels-corporation-cutback']}

In [ ]:
featureX = feature_list[1]
featureX1 = feature_list[12]
sub_featureX = 'y'
sub_featureX1 = 'y'
data = training_data
class_labels = data['class'].unique()
data_split = data[(data['class'] == class_labels[0])].reset_index(drop = True)
p = Prob_X_condition_X1_Y(data_split, featureX, featureX1, sub_featureX, sub_featureX1)
print (p)

In [101]:
def calling_model(selection):
    if (selection == 'n'):
        #Calling the Naive Bayes algorithm.
        Training_naive(training_data)
        Prediction_naive(testing_data)
    if (selection == 't'):
        #Calling the Bayesian Network.
        weight = Weight_collection(training_data)
        tree = MST(weight)
        #pd.DataFrame(weight)

In [104]:
calling_model('t')

In [26]:
#Reading the Chess-King dataset.
raw_data = arff.load(open('chess-KingRookVKingPawn.arff')) # Loading the training datasets
# Creating a list of the features
feature_list = [] 
for x in raw_data['attributes']:
    feature_list.append(x[0])
training_data = pd.DataFrame(np.array(raw_data['data']), columns = feature_list) # Saving as a Pandas Dataframe
training_data = training_data.apply(pd.to_numeric, errors='ignore') # Converting the releavent columns to numeric

In [170]:
def stratification(data, n):
    #n-fold cross validation.
    # Creating a list of the features
    feature_list = [] 
    for x in raw_data['attributes']:
        feature_list.append(x[0])    
    #Create a empty dataframe to contain the strata of the data.
    stratified_data = pd.DataFrame(columns=feature_list)
    stratified_data['Fold_of_instance'] = np.nan
    #Find number of data-points to go into each strata.
    data_count = data['class'].value_counts()
    won_data_count = int(data_count['won']/n)
    nowin_data_count = int(data_count['nowin']/n)
    #Create two partition of the dataset (won and nowin).
    won_data = data[data['class'] == 'won'].reset_index(drop = 'True')
    nowin_data = data[data['class'] == 'nowin'].reset_index(drop = 'True')
    #Appending the split data into a new dataframe with the strata number.
    for i in range(0, n-1):
        #Appending for won data.
        index_for_won_data = np.random.choice(won_data.index.values, won_data_count, replace = False)
        won_dataframe  = won_data.iloc[index_for_won_data]
        won_data = won_data[~won_data.isin(won_dataframe)].dropna()
        won_data = won_data.reset_index(drop = True)
        won_dataframe['Fold_of_instance'] = i + 1
        stratified_data = stratified_data.append(won_dataframe, ignore_index=True)
        #Appending for nowin data.
        index_for_nowin_data = random.choice(nowin_data.index.values, nowin_data_count, replace = False)
        nowin_dataframe  = nowin_data.iloc[index_for_nowin_data]
        nowin_data = nowin_data[~nowin_data.isin(nowin_dataframe)].dropna()
        nowin_data = nowin_data.reset_index(drop = True)
        nowin_dataframe['Fold_of_instance'] = i + 1
        stratified_data = stratified_data.append(nowin_dataframe, ignore_index=True)
    #We might have ignored certain values due to roundin off. To consider all the remaining values.
    #Appending for won data.
    index_for_won_data = np.random.choice(won_data.index.values, len(won_data), replace = False)
    won_dataframe  = won_data.iloc[index_for_won_data]
    won_data = won_data[~won_data.isin(won_dataframe)].dropna()
    won_data = won_data.reset_index(drop = True)
    won_dataframe['Fold_of_instance'] = n
    stratified_data = stratified_data.append(won_dataframe, ignore_index=True)
    #Appending for nowin data.
    index_for_nowin_data = random.choice(nowin_data.index.values, len(nowin_data), replace = False)
    nowin_dataframe  = nowin_data.iloc[index_for_nowin_data]
    nowin_data = nowin_data[~nowin_data.isin(nowin_dataframe)].dropna()
    nowin_data = nowin_data.reset_index(drop = True)
    nowin_dataframe['Fold_of_instance'] = n
    stratified_data = stratified_data.append(nowin_dataframe, ignore_index=True)
    #Returning the n - fold dataset.
    return stratified_data

In [ ]:
# Main output for Chess-King-Rook.
# Creating the stratified dataset
n = 10
stratified_training_data = stratification (training_data, n)
for iteration in range(n):
    #Training over each stratified dataset.
    substratified_training_data = stratified_training_data[stratified_training_data['Fold_of_instance'] != (iteration + 1)]
    substratified_training_data = substratified_training_data.reset_index(drop = True)
    batch_size = substratified_training_data['class'].value_counts()
    training_king = substratified_training_data.iloc[:, 0:len(stratified_training_data.columns)-1]   
    Training(training_king)
#Working on testing dataset.
substratified_testing_data = stratified_training_data[stratified_training_data['Fold_of_instance'] == (iteration + 1)]
substratified_testing_data = substratified_testing_data.reset_index(drop = True)
testing_king = substratified_testing_data .iloc[:, 0:len(stratified_training_data.columns)-1]
#Prediction.
Prediction_naive(testing_king)